## Treating missing values, winsorizing outliers and feature normalization

In [2]:
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf


# loading packages
import numpy as np
import pandas as pd


# data visualization and missing values
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
sns.set()
import missingno as msno # missing values
%matplotlib inline

# stats
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
df = pd.read_csv(r"C:\Users\Besitzer\Documents\Data\ger_factor_data_short.csv", dtype ={"comp_tpci": str})
df["date"] = pd.to_datetime(df["date"])  #convert to date format
df["eom"] = pd.to_datetime(df["eom"])    #convert to date format
df.head()

,id,date,eom,size_grp,primary_sec,curcd,fx,common,comp_tpci,comp_exchg,...,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,comp_001166_02W,1999-07-30,1999-07-31,small,0,EUR,1.070296,1,0,154,...,NaN,-0.661029,-0.047722,0.140976,NaN,NaN,NaN,NaN,NaN,NaN
1,comp_001166_02W,1999-08-31,1999-08-31,small,0,EUR,1.056227,1,0,154,...,NaN,-0.661029,-0.047722,0.140976,NaN,NaN,NaN,NaN,NaN,NaN
2,comp_001166_02W,1999-09-30,1999-09-30,small,0,EUR,1.064990,1,0,154,...,NaN,-0.661029,-0.047722,0.140976,NaN,NaN,NaN,NaN,NaN,NaN
3,comp_001166_02W,1999-10-29,1999-10-31,small,0,EUR,1.051050,1,0,154,...,NaN,-0.398598,0.093191,0.045857,NaN,NaN,NaN,NaN,NaN,NaN
4,comp_001166_02W,1999-11-30,1999-11-30,small,0,EUR,1.006891,1,0,154,...,NaN,-0.398598,0.093191,0.045857,NaN,NaN,NaN,NaN,NaN,NaN


### Missing values

- Inspect the amount of missing values per variable:

In [11]:
pd.set_option('display.max_rows',None)
df.isna().sum()

id                           0
date                         0
eom                          0
size_grp                  6256
primary_sec                  0
curcd                        0
fx                           0
common                       0
comp_tpci                    0
comp_exchg                   0
sic                      32512
shares                    6256
me_company                6256
prc_local                    0
dolvol                   68758
ret                          0
ret_local                    0
ret_exc                      0
ret_lag_dif                  0
ret_exc_lead1m            4750
enterprise_value         41189
book_equity               9412
assets                       0
sales                     1930
net_income                  11
bidask                       0
prc_high                     0
prc_low                      0
tvol                     68758
niq_su                  159582
ret_6_1                  40467
ret_12_1                 53911
saleq_su

- Drop observations without SIC-codes

In [12]:
df = df.dropna( how='any',subset=["sic"])

- Replace the missing valuues with the cross-sectional median at each month


In [13]:
df = df.fillna(df.groupby('eom').transform('median'))

- Drop variable characteristics the more than 10,000 missing values:


In [14]:
pd.set_option('display.max_rows',None)
df.isna().sum().sort_values(ascending = False)

seas_16_20na            98269
seas_16_20an            98098
ivol_hxz4_21d           59487
iskew_hxz4_21d          59487
seas_11_15na            44474
seas_11_15an            44474
seas_6_10na             14543
seas_6_10an             14543
capex_abn               11828
zero_trades_252d        10056
turnover_126d            9341
dolvol_126d              9341
dolvol_var_126d          9341
ami_126d                 9341
zero_trades_126d         9341
turnover_var_126d        9341
qmj_growth               9341
qmj                      9341
tvol                     8869
dolvol                   8869
zero_trades_21d          8869
ni_inc8q                 7929
capx_gr2                 7630
capx_gr1                 6955
size_grp                 6198
niq_be_chg1              6171
niq_at_chg1              6171
niq_su                   6083
capx_gr3                 5932
resff3_12_1              4959
resff3_6_1               4959
niq_be                   3955
niq_at                   3445
ocfq_saleq

In [15]:
df.drop(["seas_16_20na", "seas_16_20an","ivol_hxz4_21d", "iskew_hxz4_21d","seas_11_15na","seas_11_15an","seas_6_10na","seas_6_10an"], axis = 1, inplace = True)

-  Drop all observations for which there are still missing values

In [16]:
df = df.dropna()

- There are 306411 observations left

In [17]:
print(len(df))


306411


In [18]:
pd.set_option('display.max_rows',None)
df.isna().sum().sort_values(ascending = False)

qmj_safety              0
netdebt_me              0
capx_gr1                0
lnoa_gr1a               0
noa_gr1a                0
noa_at                  0
ppeinv_gr1a             0
at_gr1                  0
capex_abn               0
eq_dur                  0
bev_mev                 0
capx_gr3                0
intrinsic_value         0
ocf_me                  0
sale_me                 0
ebitda_mev              0
sale_gr1                0
sale_gr3                0
eqnpo_me                0
eqpo_me                 0
capx_gr2                0
chcsho_12m              0
fcf_me                  0
ncoa_gr1a               0
taccruals_ni            0
oaccruals_ni            0
be_gr1a                 0
fnl_gr1a                0
lti_gr1a                0
sti_gr1a                0
nfna_gr1a               0
ncol_gr1a               0
nncoa_gr1a              0
eqnpo_12m               0
col_gr1a                0
coa_gr1a                0
cowc_gr1a               0
taccruals_at            0
oaccruals_at

- Save cleaned data set

In [19]:
df.to_csv(r'C:\Users\Besitzer\Documents\Data\ger_factor_data_without_NAN.csv', index = False)

In [57]:
df = pd.read_csv(r"C:\Users\Besitzer\Documents\Data\ger_factor_data_without_NAN.csv", dtype ={"comp_tpci": str})
df["date"] = pd.to_datetime(df["date"])  #convert to date format
df["eom"] = pd.to_datetime(df["eom"])    #convert to date format
df.head()

,id,date,eom,size_grp,primary_sec,curcd,fx,common,comp_tpci,comp_exchg,...,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,comp_001166_02W,1999-07-30,1999-07-31,small,0,EUR,1.070296,1,0,154,...,0.964529,-0.661029,-0.047722,0.140976,0.503758,0.514521,0.000000,0.019932,0.052354,-0.130143
1,comp_001166_02W,1999-08-31,1999-08-31,small,0,EUR,1.056227,1,0,154,...,0.894882,-0.661029,-0.047722,0.140976,0.511799,0.506162,-0.008770,0.019840,0.043850,-0.118398
2,comp_001166_02W,1999-09-30,1999-09-30,small,0,EUR,1.064990,1,0,154,...,0.880642,-0.661029,-0.047722,0.140976,0.502374,0.505931,-0.008770,0.011904,0.043850,-0.125760
3,comp_001166_02W,1999-10-29,1999-10-31,small,0,EUR,1.051050,1,0,154,...,0.952324,-0.398598,0.093191,0.045857,0.512833,0.509351,-0.008998,0.015945,0.044988,-0.111005
4,comp_001166_02W,1999-11-30,1999-11-30,small,0,EUR,1.006891,1,0,154,...,1.027418,-0.398598,0.093191,0.045857,0.506444,0.517646,0.026716,0.033999,0.080147,-0.065429


## Explore the data before treating 


In [20]:
df = df.sort_values(by=['eom', "id"])

In [19]:
# The observation period contains 300 months in the period from 04.1995 to 12.2020'
df.head(5).append(df.tail(5))

,id,date,eom,size_grp,primary_sec,curcd,fx,common,comp_tpci,comp_exchg,...,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
3557,comp_011217_05W,1995-04-28,1995-04-30,mega,0,DEM,0.723476,1,0,163,...,1.502913,0.278979,0.088404,0.037669,0.500384,0.499886,0.018878,-0.024001,0.000000,0.029671
7008,comp_015334_02W,1995-04-28,1995-04-30,mega,0,DEM,0.723476,1,0,154,...,1.271648,0.187998,0.151067,0.129397,0.500384,0.499886,0.018878,-0.024001,0.000000,0.029671
7411,comp_015496_01W,1995-04-28,1995-04-30,small,0,DEM,0.723476,1,0,154,...,1.061040,0.087750,0.006923,-0.001141,0.500384,0.499886,0.018878,-0.024001,0.000000,0.029671
7627,comp_015496_02W,1995-04-28,1995-04-30,large,1,DEM,0.723476,1,0,154,...,1.266525,0.087750,0.006923,-0.001141,0.232975,0.538805,0.773998,-0.424014,0.906144,0.830793
7668,comp_015518_01W,1995-04-28,1995-04-30,large,1,DEM,0.723476,1,0,154,...,1.066115,0.054419,0.013456,0.000189,0.623656,0.446684,0.509706,-0.728023,0.868388,0.815957
306352,comp_333391_05W,2020-12-30,2020-12-31,nano,0,EUR,1.228850,1,0,154,...,0.577215,-0.267562,-0.146800,0.642355,0.504174,0.509169,0.029274,0.030414,0.048699,0.062550
306370,comp_333885_01W,2020-12-30,2020-12-31,large,1,EUR,1.228850,1,0,171,...,0.961956,0.639671,0.193740,0.074673,0.664996,0.382121,0.029274,1.618001,0.048699,-0.446789
306384,comp_334036_02W,2020-12-30,2020-12-31,nano,0,EUR,1.228850,1,0,154,...,1.713296,-1.335185,-0.366940,0.061961,0.504174,0.509169,0.029274,0.030414,0.048699,0.062550
306389,comp_334302_02W,2020-12-30,2020-12-31,micro,0,EUR,1.228850,1,0,257,...,1.024812,0.251130,0.075040,-0.115848,0.504174,0.509169,0.029274,0.030414,0.048699,0.062550
306410,comp_336873_02W,2020-12-30,2020-12-31,nano,0,EUR,1.228850,1,0,154,...,1.696445,0.056421,-0.379762,-0.467364,0.504174,0.509169,0.029274,0.030414,0.048699,0.062550


In [50]:
df["eom"].unique()

array(['1995-04-30T00:00:00.000000000', '1995-05-31T00:00:00.000000000',
       '1995-06-30T00:00:00.000000000', '1996-04-30T00:00:00.000000000',
       '1996-05-31T00:00:00.000000000', '1996-06-30T00:00:00.000000000',
       '1996-07-31T00:00:00.000000000', '1996-08-31T00:00:00.000000000',
       '1996-09-30T00:00:00.000000000', '1996-10-31T00:00:00.000000000',
       '1996-11-30T00:00:00.000000000', '1996-12-31T00:00:00.000000000',
       '1997-01-31T00:00:00.000000000', '1997-02-28T00:00:00.000000000',
       '1997-03-31T00:00:00.000000000', '1997-04-30T00:00:00.000000000',
       '1997-05-31T00:00:00.000000000', '1997-06-30T00:00:00.000000000',
       '1997-07-31T00:00:00.000000000', '1997-08-31T00:00:00.000000000',
       '1997-09-30T00:00:00.000000000', '1997-10-31T00:00:00.000000000',
       '1997-11-30T00:00:00.000000000', '1997-12-31T00:00:00.000000000',
       '1998-01-31T00:00:00.000000000', '1998-02-28T00:00:00.000000000',
       '1998-03-31T00:00:00.000000000', '1998-04-30

In [ ]:
#start from 1997--> first year with complete data for all months

In [58]:
df = df[~(df['eom'] < '1997-01-31')]

In [59]:
#the new observation period contains 24 years (288 Months) form 01.1997 to 12.2020 
print(df['eom'].nunique())

288


In [38]:
df.head(5).append(df.tail(5))

,id,date,eom,size_grp,primary_sec,curcd,fx,common,comp_tpci,comp_exchg,...,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
3149,comp_010787_02W,1997-01-31,1997-01-31,mega,0,EUR,1.196431,1,0,154,...,1.419911,0.151126,0.091640,-0.019869,0.531032,0.492101,-0.025660,0.068057,-0.042767,-0.056583
3569,comp_011217_05W,1997-01-31,1997-01-31,mega,0,DEM,0.610616,1,0,163,...,1.544091,0.229582,0.047085,-0.045487,0.531032,0.492101,-0.025660,0.068057,-0.042767,-0.056583
7020,comp_015334_02W,1997-01-31,1997-01-31,mega,0,DEM,0.610616,1,0,154,...,1.682550,0.185862,0.114406,-0.036662,0.531032,0.492101,-0.025660,0.068057,-0.042767,-0.056583
7423,comp_015496_01W,1997-01-31,1997-01-31,small,0,DEM,0.610616,1,0,154,...,1.465363,0.096171,0.004535,-0.002389,0.531032,0.492101,-0.025660,0.068057,-0.042767,-0.056583
7639,comp_015496_02W,1997-01-31,1997-01-31,large,1,DEM,0.610616,1,0,154,...,1.253663,0.096171,0.004535,-0.002389,0.531032,0.513601,-0.111194,-0.469594,-0.145407,0.603547
306352,comp_333391_05W,2020-12-30,2020-12-31,nano,0,EUR,1.228850,1,0,154,...,0.577215,-0.267562,-0.146800,0.642355,0.504174,0.509169,0.029274,0.030414,0.048699,0.062550
306370,comp_333885_01W,2020-12-30,2020-12-31,large,1,EUR,1.228850,1,0,171,...,0.961956,0.639671,0.193740,0.074673,0.664996,0.382121,0.029274,1.618001,0.048699,-0.446789
306384,comp_334036_02W,2020-12-30,2020-12-31,nano,0,EUR,1.228850,1,0,154,...,1.713296,-1.335185,-0.366940,0.061961,0.504174,0.509169,0.029274,0.030414,0.048699,0.062550
306389,comp_334302_02W,2020-12-30,2020-12-31,micro,0,EUR,1.228850,1,0,257,...,1.024812,0.251130,0.075040,-0.115848,0.504174,0.509169,0.029274,0.030414,0.048699,0.062550
306410,comp_336873_02W,2020-12-30,2020-12-31,nano,0,EUR,1.228850,1,0,154,...,1.696445,0.056421,-0.379762,-0.467364,0.504174,0.509169,0.029274,0.030414,0.048699,0.062550


In [60]:
#There are 2660 unique german stocks
print(df['id'].nunique())

2660


In [61]:
#There are 370 unique SIC-codes
df['sic'].nunique()

370

In [90]:
#Inspect variable types
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306411 entries, 3557 to 306410
Data columns (total 174 columns):
 #   Column                Dtype         
---  ------                -----         
 0   id                    object        
 1   date                  datetime64[ns]
 2   eom                   datetime64[ns]
 3   size_grp              object        
 4   primary_sec           int64         
 5   curcd                 object        
 6   fx                    float64       
 7   common                int64         
 8   comp_tpci             object        
 9   comp_exchg            int64         
 10  sic                   float64       
 11  shares                float64       
 12  me_company            float64       
 13  prc_local             float64       
 14  dolvol                float64       
 15  ret                   float64       
 16  ret_local             float64       
 17  ret_exc               float64       
 18  ret_lag_dif           int64         
 19

### Treating Outliers

All monthly firm characteristics are winsorized at the 1% and 99% levels to correct for outliers. In contrast to Gu, Kelly, and Xiu (2020), the excess returns are also winsorized.

In [62]:
features = df.columns[11:174].tolist()
features

['shares',
 'me_company',
 'prc_local',
 'dolvol',
 'ret',
 'ret_local',
 'ret_exc',
 'ret_lag_dif',
 'ret_exc_lead1m',
 'enterprise_value',
 'book_equity',
 'assets',
 'sales',
 'net_income',
 'bidask',
 'prc_high',
 'prc_low',
 'tvol',
 'niq_su',
 'ret_6_1',
 'ret_12_1',
 'saleq_su',
 'tax_gr1a',
 'ni_inc8q',
 'prc_highprc_252d',
 'resff3_6_1',
 'resff3_12_1',
 'be_me',
 'debt_me',
 'at_me',
 'ret_60_12',
 'ni_me',
 'fcf_me',
 'div12m_me',
 'eqpo_me',
 'eqnpo_me',
 'sale_gr3',
 'sale_gr1',
 'ebitda_mev',
 'sale_me',
 'ocf_me',
 'intrinsic_value',
 'bev_mev',
 'netdebt_me',
 'eq_dur',
 'capex_abn',
 'at_gr1',
 'ppeinv_gr1a',
 'noa_at',
 'noa_gr1a',
 'lnoa_gr1a',
 'capx_gr1',
 'capx_gr2',
 'capx_gr3',
 'chcsho_12m',
 'eqnpo_12m',
 'debt_gr3',
 'inv_gr1',
 'inv_gr1a',
 'oaccruals_at',
 'taccruals_at',
 'cowc_gr1a',
 'coa_gr1a',
 'col_gr1a',
 'nncoa_gr1a',
 'ncoa_gr1a',
 'ncol_gr1a',
 'nfna_gr1a',
 'sti_gr1a',
 'lti_gr1a',
 'fnl_gr1a',
 'be_gr1a',
 'oaccruals_ni',
 'taccruals_ni',
 'neti

In [63]:
df[features] = df[features].apply(lambda x: x.clip(*x.quantile([0.01, 0.99])))

In [64]:
with pd.option_context('display.max_columns', 174):
    print(df.describe(include='all'))

<ipython-input-64-f07716cb0f4d>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(df.describe(include='all'))
<ipython-input-64-f07716cb0f4d>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(df.describe(include='all'))


                     id                 date                  eom size_grp  \
count            302640               302640               302640   302640   
unique             2660                 4862                  288        5   
top     comp_200759_02W  2014-12-30 00:00:00  2014-10-31 00:00:00    micro   
freq                288                 1250                 1411   100999   
first               NaN  1997-01-02 00:00:00  1997-01-31 00:00:00      NaN   
last                NaN  2020-12-30 00:00:00  2020-12-31 00:00:00      NaN   
mean                NaN                  NaN                  NaN      NaN   
std                 NaN                  NaN                  NaN      NaN   
min                 NaN                  NaN                  NaN      NaN   
25%                 NaN                  NaN                  NaN      NaN   
50%                 NaN                  NaN                  NaN      NaN   
75%                 NaN                  NaN                  Na

### Normalization

In [65]:
df[features]=sklearn.preprocessing.minmax_scale(df[features], feature_range=(0, 1), axis=0, copy=False)

In [67]:
df.head()

,id,date,eom,size_grp,primary_sec,curcd,fx,common,comp_tpci,comp_exchg,...,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,comp_001166_02W,1999-07-30,1999-07-31,small,0,EUR,1.070296,1,0,154,...,0.266162,0.702730,0.619276,0.609128,0.522291,0.595222,0.498247,0.505300,0.515193,0.458964
1,comp_001166_02W,1999-08-31,1999-08-31,small,0,EUR,1.056227,1,0,154,...,0.236868,0.702730,0.619276,0.609128,0.531882,0.583406,0.495584,0.505273,0.512611,0.462543
2,comp_001166_02W,1999-09-30,1999-09-30,small,0,EUR,1.064990,1,0,154,...,0.230879,0.702730,0.619276,0.609128,0.520640,0.583079,0.495584,0.502896,0.512611,0.460300
3,comp_001166_02W,1999-10-29,1999-10-31,small,0,EUR,1.051050,1,0,154,...,0.261028,0.760038,0.747463,0.524785,0.533115,0.587913,0.495515,0.504106,0.512957,0.464796
4,comp_001166_02W,1999-11-30,1999-11-30,small,0,EUR,1.006891,1,0,154,...,0.292613,0.760038,0.747463,0.524785,0.525494,0.599639,0.506357,0.509513,0.523632,0.478686


In [68]:
df["me_company"].min()
df["me_company"].max()

0.9999999999999999

### Convert float64 to float32 

In [69]:
for column in df:
    if df[column].dtypes == "float64": 
        df[column] = df[column].astype("float32")

In [70]:
#Inspect variable types
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302640 entries, 0 to 306410
Data columns (total 174 columns):
 #   Column                Dtype         
---  ------                -----         
 0   id                    object        
 1   date                  datetime64[ns]
 2   eom                   datetime64[ns]
 3   size_grp              object        
 4   primary_sec           int64         
 5   curcd                 object        
 6   fx                    float32       
 7   common                int64         
 8   comp_tpci             object        
 9   comp_exchg            int64         
 10  sic                   float32       
 11  shares                float32       
 12  me_company            float32       
 13  prc_local             float32       
 14  dolvol                float32       
 15  ret                   float32       
 16  ret_local             float32       
 17  ret_exc               float32       
 18  ret_lag_dif           float32       
 19  r

In [71]:
df.to_csv(r'C:\Users\Besitzer\Documents\Data\ger_factor_data_win_norm.csv', index = False)

### Create a new lead variable for excess return

In [72]:
df['ret_exc_l1'] = df.groupby(['id'])['ret_exc'].shift(-1)

In [73]:
df.head()

,id,date,eom,size_grp,primary_sec,curcd,fx,common,comp_tpci,comp_exchg,...,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety,ret_exc_l1
0,comp_001166_02W,1999-07-30,1999-07-31,small,0,EUR,1.070296,1,0,154,...,0.702730,0.619276,0.609128,0.522291,0.595222,0.498247,0.505300,0.515193,0.458964,0.343650
1,comp_001166_02W,1999-08-31,1999-08-31,small,0,EUR,1.056227,1,0,154,...,0.702730,0.619276,0.609128,0.531882,0.583405,0.495584,0.505273,0.512611,0.462543,0.605915
2,comp_001166_02W,1999-09-30,1999-09-30,small,0,EUR,1.064990,1,0,154,...,0.702730,0.619276,0.609128,0.520640,0.583079,0.495584,0.502896,0.512611,0.460300,0.382962
3,comp_001166_02W,1999-10-29,1999-10-31,small,0,EUR,1.051050,1,0,154,...,0.760038,0.747463,0.524785,0.533115,0.587913,0.495515,0.504106,0.512957,0.464796,1.000000
4,comp_001166_02W,1999-11-30,1999-11-30,small,0,EUR,1.006891,1,0,154,...,0.760038,0.747463,0.524785,0.525494,0.599639,0.506357,0.509513,0.523632,0.478686,0.929363


In [75]:
#There are 2660 unique german stocks --> We can see that we have no previous value that we can use to predict the frst value in the sequence. 
df['ret_exc_l1'].isna().sum()

2660

In [79]:
df.to_csv(r'C:\Users\Besitzer\Documents\Data\ger_factor_data_win_norm.csv', index = False)